#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [11]:
def fbprobetModel(stock_name,input_data):
    # Load the saved model from the file
    loaded_model = {}
    with open('../TrainedModel/prophet/{}_prophet_model_1.pkl'.format(stock_name), 'rb') as f:
        loaded_model = pickle.load(f)
    
    # Load the saved model from the file
    loaded_model_prophet = {}
    with open('../TrainedModel/prophet/{}prophet_model_2.pkl'.format(stock_name), 'rb') as f:
        loaded_model_prophet = pickle.load(f)
    
    data = {
    'ds' : [input_data["ds"]]
     }
  

    # Create a new DataFrame object
    data_fr = pd.DataFrame(data)
    data_fr.head()
    
    forecast = loaded_model_prophet.predict(data_fr)
    
    #get some variables
    df_xgb = forecast.loc[:, ["trend", "weekly",  "multiplicative_terms","yhat"]]
    # Create a dictionary with data
    data = {
     #   'Open': [input_data["Open"]],

        'High': [input_data["High"]],
        'Low': [input_data["Low"]],
        'y': [input_data["Close"]],
        'Volume': [input_data["Volume"]],
        'trend': df_xgb.head(1).trend.values,
        'weekly':df_xgb.head(1).weekly.values,
        'multiplicative_terms': df_xgb.head(1).multiplicative_terms.values,
    }
    print("point 1",data)


    # Create a new DataFrame object
    df = pd.DataFrame(data)
    print("point 2",df.head())

    Future = xgb.DMatrix(df, label = df.y)
    print("point 3",df)

    f_predictions = pd.Series(loaded_model.predict(Future), name = "XGBoost")
    return f_predictions


In [12]:
def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    return data

In [13]:
tickers = ["OLECTRA.NS","LT.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI"]
  # Add the tickers you want to fetch data for
stock_data = fetch_stock_data(tickers)


[*********************100%***********************]  8 of 8 completed


In [5]:
stock_data

Adj Close                                                   \
              BEL.NS   CONCOR.NS ELGIEQUIP.NS     IOC.NS        LT.NS   
Date                                                                    
2023-06-30    125.75  661.799988   541.599976  91.300003  2475.550049   

                                                    Close              ...  \
            OLECTRA.NS TATAELXSI.NS         ^NSEI  BEL.NS   CONCOR.NS  ...   
Date                                                                   ...   
2023-06-30  972.200012       7588.5  19189.050781  125.75  661.799988  ...   

                   Open                  Volume                         \
           TATAELXSI.NS         ^NSEI    BEL.NS CONCOR.NS ELGIEQUIP.NS   
Date                                                                     
2023-06-30       7623.0  19076.849609  27965021    580733       511615   

                                                               
              IOC.NS    LT.NS OLECTRA.NS TATAELXSI.NS   ^NSEI  
Date                                                           
2023-06-30  12581625  2690469    3391180       139310  247900  

[1 rows x 48 columns]

In [16]:

result = []
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5

current_date = datetime.now().strftime("%d-%m-%Y")

for ticker in tickers:
    stock_name = ticker
    if ticker == "^NSEI":
        stock_name = "NSEI"
        
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],

                    "Close" : stock_data.head(1).High[ticker].values[0],
                    "High" : stock_data.head(1).High[ticker].values[0],
                    "Low" : stock_data.head(1).Low[ticker].values[0], 
                    "Volume" : stock_data.head(1).Volume[ticker].values[0],
                    "ds":np.datetime_as_string(stock_data.index, unit='D')[0]
                }
    
   
    stock_name_only = stock_name.replace(".NS","")
    print(stock_name_only)
    print(input_data)

        
    model_result  = fbprobetModel(stock_name_only,input_data)
    print(model_result)
    result.append([current_date,model_result[0],0,'prophet',stock_name_only,stock_data.head(1).Close[ticker].values[0]])


OLECTRA
{'Close': 1006.8499755859375, 'High': 1006.8499755859375, 'Low': 960.0, 'Volume': 3391180, 'ds': '2023-06-30'}
point 1 {'High': [1006.8499755859375], 'Low': [960.0], 'y': [1006.8499755859375], 'Volume': [3391180], 'trend': array([218.97741562]), 'weekly': array([2.12059829]), 'multiplicative_terms': array([2.06659652])}
point 2           High    Low            y   Volume       trend    weekly  \
0  1006.849976  960.0  1006.849976  3391180  218.977416  2.120598   

   multiplicative_terms  
0              2.066597  
point 3           High    Low            y   Volume       trend    weekly  \
0  1006.849976  960.0  1006.849976  3391180  218.977416  2.120598   

   multiplicative_terms  
0              2.066597  
0    971.651428
Name: XGBoost, dtype: float32
LT
{'Close': 2483.5, 'High': 2483.5, 'Low': 2415.050048828125, 'Volume': 2690469, 'ds': '2023-06-30'}
point 1 {'High': [2483.5], 'Low': [2415.050048828125], 'y': [2483.5], 'Volume': [2690469], 'trend': array([1605.49455423]), 

In [3]:
loaded_model = {}
with open('../TrainedModel/prophet/{}_prophet_model_1.pkl'.format("IOC"), 'rb') as f:
    loaded_model = pickle.load(f)


In [5]:
print(loaded_model.add_country_holidays(country_name='USA')
)

AttributeError: 'Booster' object has no attribute 'add_country_holidays'

In [19]:
dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock]})
    
dataa

{'data': [{'cdate': '02-07-2023',
   'result': '971.6514',
   'pre_close': '972.2000122070312',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'OLECTRA'},
  {'cdate': '02-07-2023',
   'result': '2470.3613',
   'pre_close': '2475.550048828125',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'LT'},
  {'cdate': '02-07-2023',
   'result': '662.8659',
   'pre_close': '661.7999877929688',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'CONCOR'},
  {'cdate': '02-07-2023',
   'result': '545.6747',
   'pre_close': '541.5999755859375',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'ELGIEQUIP'},
  {'cdate': '02-07-2023',
   'result': '91.81135',
   'pre_close': '91.30000305175781',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'IOC'},
  {'cdate': '02-07-2023',
   'result': '125.674614',
   'pre_close': '125.75',
   'isClassification': 0,
   'model': 'prophet',
   'stock': 'BEL'},
  {'cdate': '02-07-2023',
   'result': '7769.5

In [20]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>
